In [30]:
%run '02_Cartan_Group.ipynb'

In [31]:
class Parabolic_Subgroup ( Algebraic_Structure ) :
    pass

In [34]:
class Parabolic_Subgroup_In_Irreducible_Cartan_Group ( Irreducible_Algebraic_Structure , Parabolic_Subgroup ) :

    def __init__( self , Parent_Group:Irreducible_Cartan_Group , Included_Nodes:set=set([]) ) -> None :
        """
        Initialize an algebraic group of given cartan type over a base field.

        INPUT:
        - ``Parent_Group`` -- Irreducible_Cartan_Group;
                The parent group G
        - ``Included_Nodes`` -- set;
                A parabolic subgroup is uniquely determined by taking the Borel subgroup B ⊂ G and adding the negative roots which are marked.

        OUTPUT: None.
        """
        assert type( Parent_Group ) == Irreducible_Cartan_Group , \
               TypeError('The parent group must be an irreducible Cartan group.')
        self._Parent_Group = Parent_Group

        assert type( Included_Nodes ) == set , \
               TypeError('The included nodes must be given as set.')
        Nodes = set( Parent_Group.Cartan_Type().index_set() )
        assert Included_Nodes.issubset( Nodes ) , \
               ValueError('The set of included nodes must be a subset of all admissible nodes ' + str(Nodes) + '.')
        self._Included_Nodes = Included_Nodes


    def __repr__ ( self ) -> ( Irreducible_Cartan_Group , set ) :
        """Returns all attributes which are necessary to initialize the object."""
        return self.Parent_Group() , self.Included_Nodes()


    def __str__ ( self ) -> str :
        """Returns a one-line string as short description."""
        return 'Parabolic subgroup P(' + str( self.Included_Nodes() ) + ') in irreducible Cartan group of type ' + self.Parent_Group().Cartan_String() + '.'


    def Cartan_Data ( self ) -> ( str , int ) :
        """Returns the Cartan data of each irreducible component of ``self``."""
        for Connected_Component in self.Connected_Components_Of_Included_Nodes() :
            Cartan_Subtype = self.Parent_Group().Cartan_Type().subtype( Connected_Component )
            yield Cartan_Subtype.type() , len(Cartan_Subtype.index_set())


    def Cartan_String ( self ) -> str :
        """Returns the Cartan string of ``self``."""
        return 'x'.join([ Cartan_Family+str(Cartan_Degree) for Cartan_Family , Cartan_Degree in self.Cartan_Data() ])


    def Cartan_Type ( self ) -> CartanType :
        """Returns the Cartan type of ``self``."""
        return CartanType( self.Cartan_String() )


    def Connected_Components_Of_Included_Nodes ( self ) -> Iterator[list]:
        """
        Returns the included nodes subdivided into lists of connected components
        (i.e. lists of nodes which are connected by edges).
        """
        return ( list( Dict.keys() ) for Dict in self.Relabelling(Decomposition_Style='Fine') )


    def Dimension ( self ) -> int :
        """
        Returns the dimension of ``self``.

        INPUT:
        - ``self`` -- ParabolicSubgroup; the parabolic subgroup P of a Cartan group G.

        OUTPUT:
        - ``Output`` -- Integer; the dimension of ``self``.

        ALGORITHM:
        Thanks to the post by Pieter Belmans concerning the dimension of partial flag varieties
        from Jun 14th, 2017 on his blog (cf. to [Blog_PieterBelmans]_). The link is
        https://pbelmans.ncag.info/blog/2017/06/14/dimensions-of-partial-flag-varieties/
        (Date: Apr 21st, 2021).

        For the Borel group B ⊂ G: dim B = # of positive roots + rank (which accounts for the center)
        For P: dim P = dim B + # of negative roots which are added to construct P

        REFERENCE:
        [Blog_PieterBelmans] https://pbelmans.ncag.info/blog/
        """
        Rank = self.Parent_Group().Cartan_Type().dynkin_diagram().rank()
        Number_Of_Positive_Roots = len( list( self.Parent_Group().Cartan_Type().root_system().root_lattice().positive_roots() ) )
        Number_Of_Available_Negative_Roots = len( self.Negative_Roots() )
        return Rank + Number_Of_Positive_Roots + Number_Of_Available_Negative_Roots


    def Excluded_Nodes ( self , Output_Type:set or list or str=set ) -> set or list or str :
        """Returns the excluded nodes."""
        Nodes = set( self.Parent_Group().Cartan_Type().index_set() )
        Output = list(Nodes.difference(self.Included_Nodes()))
        Output.sort()
        if Output_Type != set :
            if   Output_Type == list : return Output
            elif Output_Type == str : return ','.join(Output)
            else : raise TypeError('The input for ``Output_Type`` is inappropriate.')
        else :
            return set(Output)


    def Included_Nodes ( self , Output_Type:set or list or str=set ) -> set or list or str :
        """Returns the attribute ``_Included_Nodes``."""
        Output = list(self._Included_Nodes)
        Output.sort()
        if Output_Type != set :
            if   Output_Type == list : return Output
            elif Output_Type == str : return ','.join(Output)
            else : raise TypeError('The input for ``Output_Type`` is inappropriate.')
        else :
            return set(Output)


    def Initialize_As_Cartan_Group ( self ) -> Irreducible_Cartan_Group or Reducible_Cartan_Group :
        """
        Returns the Cartan group associated to the Cartan string of ``self``.
        Thus, one has access to the methods of the class ``Cartan_Group`` and its subclasses.
        """
        Components = [ Irreducible_Cartan_Group( Cartan_Family , Cartan_Degree ) for Cartan_Family , Cartan_Degree in self.Cartan_Data() ]
        if len(Components) == 1 : return Components[0]
        else                    : return Direct_Sum_Of_Cartan_Groups( Components )


    # Synonym for the method ``Is_Minimal``
    def Is_Borel ( self ) -> bool :
        """Test if ``self`` is Borel (i.e. it is minimal)."""
        return self.Is_Minimal()


    def Is_Complete ( self ) -> bool :
        """Test if ``self`` is complete (i.e. all nodes are included; or equivalently, no node is excluded)."""
        return len( self.Excluded_Nodes(Output_Type=list) ) == 0


    def Is_Maximal ( self ) -> bool :
        """Test if ``self`` is maximal (i.e. almost all nodes are included; or equivalently, there is only one node excluded)."""
        return len( self.Excluded_Nodes(Output_Type=list) ) == 1


    def Is_Minimal ( self ) -> bool :
        """Test if ``self`` is minimal (i.e. no node is included; or equivalently, all nodes are excluded)."""
        return len( self.Included_Nodes(Output_Type=list) ) == 0


    # Synonym for the method ``Is_Complete``
    def Is_Parent_Group ( self ) -> bool :
        """Test if ``self`` coincides with the parent group (i.e. it is complete)."""
        return self.Is_Complete()


    def Negative_Roots ( self ) -> list :
        """Returns a list of all negative roots which are available for ``self``."""
        return [ Negative_Root for Negative_Root in list( self.Parent_Group().Cartan_Type().root_system().root_lattice().negative_roots() )
                 if not False in [ Node in self.Included_Nodes() for Node , Coefficient in Negative_Root ]
               ]


    def Parent_Group ( self ) -> Irreducible_Cartan_Group :
        """Returns the attribute ``_Parent_Group``."""
        return self._Parent_Group


    def Relabelling ( self , Decomposition_Style:'Fine' or 'Coarse'='Coarse' ) -> dict or list[dict] :
        """Returns the relabelling of the included nodes with respect to the parent group."""
        if   Decomposition_Style == 'Coarse' :
            return { Included_Node : Counter for Counter , Included_Node in enumerate( self.Included_Nodes(Output_Type=list) , start=1 ) }

        elif Decomposition_Style == 'Fine' :
            Connected_Components = []
            Included_Nodes = self.Included_Nodes(Output_Type=list)
            Cartan_Family = self.Parent_Group().Cartan_Family()
            Cartan_Degree = self.Parent_Group().Cartan_Degree()
            End = Cartan_Degree

            # For the case "Cartan_Family == E", the Dynkin-diagram-ordering is sligthly is different from a lexicographical-ordering.
            # In particluar, th second node is onyl one-connected to the fourth one and NOT between the third and fourth one.
            if not Cartan_Family == 'E' :
                Included_Nodes_In_Order_As_Given_By_Dynkin_Diagram = Included_Nodes
            else :
                Included_Nodes_In_Order_As_Given_By_Dynkin_Diagram = [ Included_Node for Included_Node in [ 1 , 3 , 4 , 2 ] + [ 5 .. End ] if Included_Node in Included_Nodes ]

            # We establish connected components of all included nodes
            for Node_Counter , Current_Node in enumerate( Included_Nodes_In_Order_As_Given_By_Dynkin_Diagram , start=1 ) :
                Is_Current_Node_In_Some_Connected_Component = False

                # The first included nodes gives arise to the first connected component ...
                if Node_Counter == 1 :
                    Connected_Components += [ [ Current_Node ] ]
                    Is_Current_Node_In_Some_Connected_Component = True

                # Is the currently considered node element of an already existing connected component?
                else :
                    if   Cartan_Family == 'D' :
                        for Connected_Component in Connected_Components :
                            if ( ( Current_Node <= End-2 and Current_Node-1 in Connected_Component ) or # k-th node is connected with the (k-1)-th node (k <= n-2)
                                 ( Current_Node == End-1 and End-1 in Connected_Component          ) or # (n-1)-th node is connected with the (n-2)-th node
                                 ( Current_Node == End   and End-2 in Connected_Component          )    # n-th node is connected with the (n-2)-th node
                               ) :
                                Connected_Component += [ Current_Node ]
                                Is_Current_Node_In_Some_Connected_Component = True
                                break

                    elif Cartan_Family == 'E' :
                        for Connected_Component in Connected_Components :
                            if ( ( Current_Node == 3 and Current_Node-2 in Connected_Component ) or # 3rd node
                                 ( Current_Node == 4 and Current_Node-1 in Connected_Component ) or # 4th node
                                 ( Current_Node == 2 and Current_Node+2 in Connected_Component ) or # 2nd node
                                 ( 5 <= Current_Node and Current_Node-1 in Connected_Component )    # 5th/ 6th/ ... node
                               ) :
                                Connected_Component += [ Current_Node ]
                                Is_Current_Node_In_Some_Connected_Component = True
                                break

                    else :
                        for Connected_Component in Connected_Components :
                            if Current_Node-1 in Connected_Component :
                                Connected_Component += [ Current_Node ]
                                Is_Current_Node_In_Some_Connected_Component = True
                                break

                # If the currently considered node seems to be not element of one of the existing connected components.
                # Hence, we establish a new connected component ...
                if not Is_Current_Node_In_Some_Connected_Component : Connected_Components += [ [ Current_Node ] ]

            Relabelling = []
            for Connected_Component in Connected_Components :
                Connected_Component.sort()
                Relabelling += [ { Included_Node : Node_Counter_In_Connected_Component for Node_Counter_In_Connected_Component , Included_Node in enumerate( Connected_Component , start=1 ) } ]
            return Relabelling

        else : raise ValueError('The input for ``Decomposition_Style`` is inappropriate.')


In [33]:
class Parabolic_Subgroup_In_Direct_Sum_Of_Cartan_Groups ( Direct_Sum_Of_Cartan_Groups , Parabolic_Subgroup ) :

    def Dimension ( self ) -> int :
        """Returns the dimension of ``self``."""
        return sum([ Component.Dimension() for Component in self.Components() ])